In [34]:
import sys
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import os
import gc
import math
import numpy as np

from collections import Counter

from util import ShowProcess
import matplotlib.pyplot as plt

import scipy.sparse as sp

## read user, ad, train, test

then merge

In [ ]:
%%time
if os.path.exists('./preliminary_contest_data/userFeature.csv'):
    print('load')
    user_feature=pd.read_csv('./preliminary_contest_data/userFeature.csv')
else:
    print('process')
    userFeature_data = []
    with open('./preliminary_contest_data/userFeature.data', 'r') as f:
        for i, line in enumerate(f):
            line = line.strip().split('|')
            userFeature_dict = {}
            for each in line:
                each_list = each.split(' ')
                userFeature_dict[each_list[0]] = ' '.join(each_list[1:])
            userFeature_data.append(userFeature_dict)
            sp.show_process()
        sp.close()
        user_feature = pd.DataFrame(userFeature_data)
        user_feature.to_csv('./preliminary_contest_data/userFeature.csv', index=False)

In [ ]:
ad_feature = pd.read_csv('./preliminary_contest_data/adFeature.csv')
train=pd.read_csv('./preliminary_contest_data/train.csv')
predict=pd.read_csv('./preliminary_contest_data/test1.csv')

In [ ]:
train.loc[train['label']==-1,'label']=0
predict['label']=-1

data=pd.concat([train,predict])
data=pd.merge(data,ad_feature,on='aid',how='left')
data=pd.merge(data,user_feature,on='uid',how='left')

## if extis read, if not save

In [4]:
%%time
if os.path.exists('./merge_data.csv'):
    print('load')
    data = pd.read_csv('./merge_data.csv')
    data = data.fillna('-1')
else:
    print('save')
    data.to_csv('./merge_data.csv')

load


In [30]:
one_hot_feature=['LBS','age','carrier','consumptionAbility','education','gender','house','os','ct',
                 'marriageStatus','advertiserId','campaignId', 'creativeId',
                 'adCategoryId', 'productId', 'productType']
vector_feature=['appIdAction','appIdInstall','interest1','interest2','interest3','interest4','interest5',
                'kw1','kw2','kw3','topic1','topic2','topic3']

### process one hot feature

In [31]:
%%time
sp = ShowProcess(len(one_hot_feature))
for feature in one_hot_feature:
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])
    sp.show_process()
sp.close()

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
done
CPU times: user 3min 14s, sys: 24.6 s, total: 3min 38s
Wall time: 4min 9s


In [49]:
%%time
enc = OneHotEncoder()
categorical_features = data[one_hot_feature].values
one_hot_features = enc.fit_transform(categorical_features)

CPU times: user 13.7 s, sys: 10.9 s, total: 24.6 s
Wall time: 30.4 s


In [162]:
type(one_hot_features), one_hot_features.shape

(scipy.sparse.csr.csr_matrix, (11064803, 1445))

### process vector feature

In [121]:
%%time
vectorize_features = []
sp = ShowProcess(len(vector_feature))
for feature in vector_feature:
    feature_df = CountVectorizer().fit_transform(data[feature])
    vectorize_features.append(feature_df)
    sp.show_process()
sp.close()

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
done
CPU times: user 14min 43s, sys: 40.6 s, total: 15min 24s
Wall time: 15min 16s


In [137]:
%%time
vectorize_features_merge = sp.hstack((vectorize_features[0],vectorize_features[1]))
for i in range(2, len(vectorize_features), 1):
    vectorize_features_merge = sp.hstack((vectorize_features_merge, vectorize_features[i]))

CPU times: user 46.9 s, sys: 48 s, total: 1min 34s
Wall time: 1min 34s


In [150]:
%%time
vectorize_features_merge = vectorize_features_merge.tocsr()

CPU times: user 27.5 s, sys: 5.02 s, total: 32.5 s
Wall time: 32.5 s


In [161]:
type(vectorize_features_merge), vectorize_features_merge.shape

(scipy.sparse.csr.csr_matrix, (11064803, 418206))

### find train, test split index

In [159]:
label = data['label']

In [204]:
%%time
train_y = label[np.where(label != -1)[0]].values
split_index = len(train_y)

CPU times: user 2.16 s, sys: 367 ms, total: 2.53 s
Wall time: 2.52 s


In [205]:
type(train_y), train_y.shape

(numpy.ndarray, (8798814,))

In [179]:
%%time
train_one_hot_matrix = one_hot_features[:split_index]
train_vectorize_matrix = vectorize_features_merge[:split_index]

test_one_hot_matrix = one_hot_features[split_index:]
test_vectorize_matrix = vectorize_features_merge[split_index:]

CPU times: user 5.02 s, sys: 8.9 s, total: 13.9 s
Wall time: 13.9 s


In [197]:
%%time
train_x = data[['creativeSize']][:split_index]
train_x = sp.csr_matrix(train_x)

train_x = sp.hstack((train_x, train_one_hot_matrix))
train_x = sp.hstack((train_x, train_vectorize_matrix))
train_x = train_x.tocsr()

CPU times: user 31.2 s, sys: 21.5 s, total: 52.7 s
Wall time: 52.7 s


In [198]:
type(train_x), train_x.shape

(scipy.sparse.csr.csr_matrix, (8798814, 419652))

In [199]:
%%time
test_x = data[['creativeSize']][split_index:]
test_x = sp.csr_matrix(test_x)

test_x = sp.hstack((test_x, test_one_hot_matrix))
test_x = sp.hstack((test_x, test_vectorize_matrix))
test_x = test_x.tocsr()

CPU times: user 7.63 s, sys: 4.48 s, total: 12.1 s
Wall time: 12.1 s


In [200]:
type(test_x), test_x.shape

(scipy.sparse.csr.csr_matrix, (2265989, 419652))

## save train_x, train_y, test_x

In [206]:
%%time
sp.save_npz('./preliminary_contest_data/train_x.npz', train_x)
sp.save_npz('./preliminary_contest_data/test_x.npz', test_x)

np.save('./preliminary_contest_data/train_y.npy', train_y)

CPU times: user 3min 25s, sys: 17.3 s, total: 3min 42s
Wall time: 3min 43s
